In [20]:
import os
import cv2
from pathlib import Path
from tqdm import tqdm

from lxml import etree
import xml.etree.ElementTree

In [42]:
classes = {'with_mask': 0, 'without_mask': 1, 'mask_weared_incorrect': 2}

In [9]:
path = Path('./annotations/')

In [10]:
xml_files = os.listdir(path)

In [12]:
len(xml_files)

853

In [23]:
if not os.path.exists('labels'):
    os.makedirs('labels')

In [43]:
for file in tqdm(xml_files):
    src_path = path / file
    
    if src_path.suffix == '.xml':
        
        root = xml.etree.ElementTree.parse(
            
            str(src_path.absolute()),
            
            parser=etree.XMLParser(encoding='utf-8')
            
        ).getroot()
        
        filename = root.find('filename').text
        
        img_path = (path.parent / 'images' / filename).with_suffix('.png')
        
        dst_path = (path.parent / 'labels' / file).with_suffix('.txt')
        with open(str(dst_path.absolute()), 'w', encoding='utf-8') as f:
        
            for object_ in root.findall('object'):

                bndbox = object_.find('bndbox')
                name = object_.find('name').text



                difficult = False
                if object_.find('difficult') is not None:
                    difficult = bool(int(object_.find('difficult').text))

                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                class_idx = classes[name]

                height, width, _ = cv2.imread(str(img_path.absolute())).shape

                center_x = (xmin + xmax) / 2 / width
                center_y = (ymin + ymax) / 2 / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                center_x = round(center_x, 6)
                center_y = round(center_y, 6)
                w = round(w, 6)
                h = round(h, 6)

                s = f'{class_idx} {center_x:.6f} {center_y:.6f} {w:.6f} {h:.6f}\n'

                f.write(s)


  1%|▍                                          | 9/853 [00:00<00:36, 23.06it/s]libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
libpng warning: iCCP: Not recognizing known sRGB profile that has been edited
100%|█████████████████████████████████████████| 853/853 [00:24<00:00, 34.15it/s]
